In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.svm import LinearSVC
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score



In [18]:
airport_data = pd.read_csv('airports.csv', delimiter=',')
airlines_data = pd.read_csv('airlines.csv', delimiter=',')
print "Completed Processing"

Completed Processing


In [19]:
data = pd.read_csv('flights.csv', delimiter=',')
print "Completed Processing"
print data.shape




/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Completed Processing
(5819079, 31)


In [4]:
print data.columns.values
print data.shape

['YEAR' 'MONTH' 'DAY' 'DAY_OF_WEEK' 'AIRLINE' 'FLIGHT_NUMBER' 'TAIL_NUMBER'
 'ORIGIN_AIRPORT' 'DESTINATION_AIRPORT' 'SCHEDULED_DEPARTURE'
 'DEPARTURE_TIME' 'DEPARTURE_DELAY' 'TAXI_OUT' 'WHEELS_OFF'
 'SCHEDULED_TIME' 'ELAPSED_TIME' 'AIR_TIME' 'DISTANCE' 'WHEELS_ON'
 'TAXI_IN' 'SCHEDULED_ARRIVAL' 'ARRIVAL_TIME' 'ARRIVAL_DELAY' 'DIVERTED'
 'CANCELLED' 'CANCELLATION_REASON' 'AIR_SYSTEM_DELAY' 'SECURITY_DELAY'
 'AIRLINE_DELAY' 'LATE_AIRCRAFT_DELAY' 'WEATHER_DELAY']
(5819079, 31)


In [21]:
#Conversion of Day, Year, Month to DATE
data['FLIGHT_DATE'] = pd.to_datetime(data[['YEAR','MONTH','DAY']])
print "COMPLETED"


COMPLETED
datetime64[ns]


In [22]:
dataDict = pd.DataFrame({'DataType': data.dtypes.values, 'Missing_values': data.isnull().sum(),'Missing%': ((data.isnull().sum())/(data.shape[0])*100)  }, index=data.columns.values)


YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4      AS             98      N407AS   
1  2015      1    1            4      AA           2336      N3KUAA   
2  2015      1    1            4      US            840      N171US   
3  2015      1    1            4      AA            258      N3HYAA   
4  2015      1    1            4      AS            135      N527AS   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE     ...       \
0            ANC                 SEA                    5     ...        
1            LAX                 PBI                   10     ...        
2            SFO                 CLT                   20     ...        
3            LAX                 MIA                   20     ...        
4            SEA                 ANC                   25     ...        

   ARRIVAL_DELAY  DIVERTED  CANCELLED  CANCELLATION_REASON  AIR_SYSTEM_DELAY  \
0          -22.0         0          0                  NaN               NaN   
1           -9.0         0          0                  NaN               NaN   
2            5.0         0          0                  NaN               NaN   
3           -9.0         0          0                  NaN               NaN   
4          -21.0         0          0                  NaN               NaN   

   SECURITY_DELAY  AIRLINE_DELAY  LATE_AIRCRAFT_DELAY  WEATHER_DELAY  \
0             NaN            NaN                  NaN            NaN   
1             NaN            NaN                  NaN            NaN   
2             NaN            NaN                  NaN            NaN   
3             NaN            NaN                  NaN            NaN   
4             NaN            NaN                  NaN            NaN   

   FLIGHT_DATE  
0   2015-01-01  
1   2015-01-01  
2   2015-01-01  
3   2015-01-01  
4   2015-01-01  

[5 rows x 32 columns]

In [23]:
#Removing numeric values in ORIGIN and DESTINATION airports
data = data[data.ORIGIN_AIRPORT.str.isnumeric() == False]
data = data[data.DESTINATION_AIRPORT.str.isnumeric() == False]
data = data[data.CANCELLED == 0]
data = data[data.DIVERTED == 0]

In [ ]:
"""def get_departure_time(time):
    global data
    HOUR = []
    MINUTE = []
    if pd.isnull(time):
        np.nan
    else:
        h = int(time)/100
        if h == 24:
            h = 0
        HOUR.append(int(h))

        m = int(time)%100
        MINUTE.append(m)
    df_hm = pd.DataFrame({'HOUR':HOUR,'MINUTE':MINUTE})
    return df_hm
#data = pd.concat((data,df_hm),axis=1)
data['DEPARTURE_TIME'].apply(lambda x: get_departure_time(x))
data.head()"""

In [59]:
"""def get_time(time):
    if pd.isnull(time):
        np.nan
    else:
        h = int(time)/100
        if h == 24:
            h = 0

        m = int(time)%100
        modified_time = datetime.time(int(h),m)
    return modified_time"""



In [24]:
def get_time(time):
    if pd.isnull(time):
        return np.nan
    else:
        if time == 2400:
            time = 0
        time = "{0:04d}".format(int(time))
        modified_time = datetime.time(int(time[0:2]), int(time[2:4]))
        return modified_time
#_____________________________________________________________________
# Function that combines a date and time to produce a datetime.datetime
def combine_date_time(dt):
    if pd.isnull(dt[0]) or pd.isnull(dt[1]):
        return np.nan
    else:
        return datetime.datetime.combine(dt[0],dt[1])
#_______________________________________________________________________________
# Function that combine two columns of the dataframe to create a datetime format
def create_flight_time(df, col):    
    delayed_time = []
    for index, cols in df[['FLIGHT_DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            delayed_time.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            delayed_time.append(combine_date_time(cols))
        else:
            cols[1] = get_time(cols[1])
            delayed_time.append(combine_date_time(cols))
    return pd.Series(delayed_time)

In [25]:
data['SCHEDULED_DEPARTURE'] = create_flight_time(data, 'SCHEDULED_DEPARTURE')
data['DEPARTURE_TIME'] = data['DEPARTURE_TIME'].apply(lambda x: get_time(x))
data['SCHEDULED_ARRIVAL'] = data['SCHEDULED_ARRIVAL'].apply(lambda x: get_time(x))
data['ARRIVAL_TIME'] = data['ARRIVAL_TIME'].apply(lambda x: get_time(x))


print "Completed"


Completed


In [26]:
data = data.drop(['YEAR','MONTH','DAY','FLIGHT_NUMBER','TAIL_NUMBER','TAXI_OUT','WHEELS_OFF','WHEELS_ON','TAXI_IN','CANCELLATION_REASON','DIVERTED','CANCELLED','WEATHER_DELAY','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY'], axis=1)
print "DONE" 

DONE


In [27]:
data.head()

DAY_OF_WEEK AIRLINE ORIGIN_AIRPORT DESTINATION_AIRPORT SCHEDULED_DEPARTURE  \
0            4      AS            ANC                 SEA 2015-01-01 00:05:00   
1            4      AA            LAX                 PBI 2015-01-01 00:10:00   
2            4      US            SFO                 CLT 2015-01-01 00:20:00   
3            4      AA            LAX                 MIA 2015-01-01 00:20:00   
4            4      AS            SEA                 ANC 2015-01-01 00:25:00   

  DEPARTURE_TIME  DEPARTURE_DELAY  SCHEDULED_TIME  ELAPSED_TIME  AIR_TIME  \
0       23:54:00            -11.0           205.0         194.0     169.0   
1       00:02:00             -8.0           280.0         279.0     263.0   
2       00:18:00             -2.0           286.0         293.0     266.0   
3       00:15:00             -5.0           285.0         281.0     258.0   
4       00:24:00             -1.0           235.0         215.0     199.0   

   DISTANCE SCHEDULED_ARRIVAL ARRIVAL_TIME  ARRIVAL_DELAY FLIGHT_DATE  
0      1448          04:30:00     04:08:00          -22.0  2015-01-01  
1      2330          07:50:00     07:41:00           -9.0  2015-01-01  
2      2296          08:06:00     08:11:00            5.0  2015-01-01  
3      2342          08:05:00     07:56:00           -9.0  2015-01-01  
4      1448          03:20:00     02:59:00          -21.0  2015-01-01

In [23]:
data.to_csv ('data.csv')


In [97]:
# Percentage of cancelled flights
(data[data.CANCELLATION_REASON.isnull() == False].DIVERTED.count())/(data.DIVERTED.count())


0.016394414010801599

In [203]:
#airline vs arrival delay
FLIGHT_ARR_DELAY_METRIC = data.loc[data['ARRIVAL_DELAY']>15,['AIRLINE','ARRIVAL_DELAY']].groupby(['AIRLINE']).agg(['count','mean','median'])
FLIGHT_ARR_DELAY_METRIC.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY','MEDIAN_DELAY']


PERCENTAGE_FLIGHTS_DELAYED = data.loc[:,['AIRLINE','ARRIVAL_DELAY']].groupby(['AIRLINE']).count().rename(columns={"ARRIVAL_DELAY":"TOTAL_FLIGHTS"})



FLIGHT_ARR_DELAY_METRIC = pd.merge(FLIGHT_ARR_DELAY_METRIC,PERCENTAGE_FLIGHTS_DELAYED, how = 'inner', left_index = True, right_index = True )
FLIGHT_ARR_DELAY_METRIC['PERCENTAGE_DELAYED'] = (FLIGHT_ARR_DELAY_METRIC.TOTAL_DELAY_COUNT/FLIGHT_ARR_DELAY_METRIC.TOTAL_FLIGHTS)*100
FLIGHT_ARR_DELAY_METRIC
#FLIGHT_ARR_DELAY_METRIC.to_csv('C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis\\AirlineVSArrivalDelay.csv')

TOTAL_DELAY_COUNT  MEAN_DELAY  MEDIAN_DELAY  TOTAL_FLIGHTS  \
AIRLINE                                                               
AA                  115542   62.093204          38.0         636554   
AS                   19615   50.331991          32.0         157025   
B6                   53405   63.132310          43.0         240304   
DL                  107560   63.497546          37.0         794817   
EV                   98932   64.797325          41.0         509621   
F9                   21711   69.587859          43.0          82005   
HA                    7489   41.578048          26.0          69815   
MQ                   56431   64.425546          43.0         257130   
NK                   31100   65.292315          44.0         105059   
OO                   97514   62.438368          41.0         528328   
UA                   96065   66.464154          43.0         462086   
US                   34930   51.926138          34.0         194223   
VX                   10497   57.857483          39.0          55813   
WN                  215470   53.945412          36.0        1138350   

         PERCENTAGE_DELAYED  
AIRLINE                      
AA                18.151170  
AS                12.491641  
B6                22.223933  
DL                13.532675  
EV                19.412858  
F9                26.475215  
HA                10.726921  
MQ                21.946486  
NK                29.602414  
OO                18.457095  
UA                20.789420  
US                17.984482  
VX                18.807446  
WN                18.928273

In [100]:
#airline vs departure delay
FLIGHT_DEP_DELAY_METRIC = data.loc[data['DEPARTURE_DELAY']>0,['AIRLINE','DEPARTURE_DELAY']].groupby(['AIRLINE']).agg(['count','mean'])
FLIGHT_DEP_DELAY_METRIC.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


PERCENTAGE_FLIGHTS_DELAYED = data.loc[:,['AIRLINE','DEPARTURE_DELAY']].groupby(['AIRLINE']).count().rename(columns={"DEPARTURE_DELAY":"TOTAL_FLIGHTS"})



FLIGHT_DEP_DELAY_METRIC = pd.merge(FLIGHT_DEP_DELAY_METRIC,PERCENTAGE_FLIGHTS_DELAYED, how = 'inner', left_index = True, right_index = True )
FLIGHT_DEP_DELAY_METRIC['PERCENTAGE_DELAYED'] = (FLIGHT_DEP_DELAY_METRIC.TOTAL_DELAY_COUNT/FLIGHT_DEP_DELAY_METRIC.TOTAL_FLIGHTS)*100
FLIGHT_DEP_DELAY_METRIC
#FLIGHT_DEP_DELAY_METRIC.to_csv(path_or_buf='C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis',mode = 'w',sep=',')

TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  \
AIRLINE                                                       
AA                  223984         34.865432         639002   
AS                   40404         26.261534         157472   
B6                   93718         38.457564         241056   
DL                  263460         30.183629         796638   
EV                  159040         41.100528         512032   
F9                   32678         45.437420          82201   
HA                   18446         17.070096          69889   
MQ                   89272         40.321859         258555   
NK                   48478         42.567350         105301   
OO                  160291         39.574992         530501   
UA                  239446         33.031301         463756   
US                   62565         28.500615         194825   
VX                   21431         30.588447          55944   
WN                  527303         27.413756        1141928   

         PERCENTAGE_DELAYED  
AIRLINE                      
AA                35.052159  
AS                25.657895  
B6                38.878103  
DL                33.071483  
EV                31.060559  
F9                39.753774  
HA                26.393281  
MQ                34.527277  
NK                46.037550  
OO                30.215023  
UA                51.631893  
US                32.113435  
VX                38.307951  
WN                46.176554

In [101]:
#Origin Airport vs Arrival Delay
AIRPORT_ARR_DELAY_METRIC = data.loc[data['ARRIVAL_DELAY']>0,['ORIGIN_AIRPORT','ARRIVAL_DELAY']].groupby(['ORIGIN_AIRPORT']).agg(['count','mean'])
AIRPORT_ARR_DELAY_METRIC.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


PERCENTAGE_FLIGHTS_DELAYED = data.loc[:,['ORIGIN_AIRPORT','ARRIVAL_DELAY']].groupby(['ORIGIN_AIRPORT']).count().rename(columns={"ARRIVAL_DELAY":"TOTAL_FLIGHTS"})



AIRPORT_ARR_DELAY_METRIC = pd.merge(AIRPORT_ARR_DELAY_METRIC,PERCENTAGE_FLIGHTS_DELAYED, how = 'inner', left_index = True, right_index = True )
AIRPORT_ARR_DELAY_METRIC['PERCENTAGE_DELAYED'] = (AIRPORT_ARR_DELAY_METRIC.TOTAL_DELAY_COUNT/AIRPORT_ARR_DELAY_METRIC.TOTAL_FLIGHTS)*100
AIRPORT_ARR_DELAY_METRIC
#AIRPORT_ARR_DELAY_METRIC.to_csv(path_or_buf='C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis',mode = 'w',sep=',')

TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  \
ORIGIN_AIRPORT                                                       
ABE                           826         36.483051           2227   
ABI                           686         39.295918           2231   
ABQ                          6965         31.850826          18918   
ABR                           246         42.211382            663   
ABY                           309         37.530744            866   
ACK                           162         56.061728            485   
ACT                           466         43.686695           1538   
ACV                           449         45.271715           1268   
ACY                          1433         37.439637           3527   
ADK                            56         26.053571             88   
ADQ                           160         21.575000            437   
AEX                          1178         41.398132           3054   
AGS                          1004         31.844622           2345   
AKN                            32         16.812500             63   
ALB                          2006         34.900299           7322   
ALO                           172         41.470930            581   
AMA                          1514         34.056803           4074   
ANC                          5196         23.359315          15818   
APN                           142         46.528169            546   
ASE                          1464         50.104508           3263   
ATL                        114872         31.051240         343506   
ATW                          1076         32.697026           2759   
AUS                         14681         34.072202          41392   
AVL                           933         39.449089           2673   
AVP                           463         40.449244           1326   
AZO                           498         39.491968           1743   
BDL                          5691         34.367071          18375   
BET                           328         25.140244            880   
BFL                           851         36.564042           2587   
BGM                            70         62.342857            259   
...                           ...               ...            ...   
SRQ                           990         40.962626           3318   
STC                            38         48.710526             77   
STL                         16803         32.371719          46181   
STT                          1358         46.034610           4171   
STX                           361         32.310249            932   
SUN                           254         36.192913            867   
SUX                           189         53.576720            589   
SWF                           260         42.019231            678   
SYR                          1592         41.640075           5447   
TLH                           957         37.819227           3141   
TOL                           220         42.918182            897   
TPA                         22072         35.625272          63077   
TRI                           631         34.166403           1906   
TTN                          1127         49.741792           2771   
TUL                          4662         34.842557          13701   
TUS                          4884         30.611179          14922   
TVC                           825         36.832727           2660   
TWF                           213         24.849765            805   
TXK                           327         31.978593            918   
TYR                           817         37.842105           2199   
TYS                          2665         40.398124           6754   
UST                            61         41.491803            144   
VEL                            23         61.173913            200   
VLD                           359         30.883008            925   
VPS                       

In [102]:
#Origin Airport vs Departure Delay
ORIGIN_AIRPORT_DEPT_DELAY_METRIC = data.loc[data['DEPARTURE_DELAY']>0,['ORIGIN_AIRPORT','DEPARTURE_DELAY']].groupby(['ORIGIN_AIRPORT']).agg(['count','mean'])
ORIGIN_AIRPORT_DEPT_DELAY_METRIC.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


PERCENTAGE_FLIGHTS_DELAYED = data.loc[:,['ORIGIN_AIRPORT','DEPARTURE_DELAY']].groupby(['ORIGIN_AIRPORT']).count().rename(columns={"DEPARTURE_DELAY":"TOTAL_FLIGHTS"})



ORIGIN_AIRPORT_DEPT_DELAY_METRIC = pd.merge(ORIGIN_AIRPORT_DEPT_DELAY_METRIC,PERCENTAGE_FLIGHTS_DELAYED, how = 'inner', left_index = True, right_index = True )
ORIGIN_AIRPORT_DEPT_DELAY_METRIC['PERCENTAGE_DELAYED'] = (ORIGIN_AIRPORT_DEPT_DELAY_METRIC.TOTAL_DELAY_COUNT/ORIGIN_AIRPORT_DEPT_DELAY_METRIC.TOTAL_FLIGHTS)*100
ORIGIN_AIRPORT_DEPT_DELAY_METRIC
#AIRPORT_ARR_DELAY_METRIC.to_csv(path_or_buf='C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis',mode = 'w',sep=',')

TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  \
DESTINATION_AIRPORT                                                       
ABE                                840         31.459524           2220   
ABI                                748         31.830214           2224   
ABQ                               7876         28.017395          18953   
ABR                                163         24.453988            657   
ABY                                347         34.138329            864   
ACK                                152         41.348684            487   
ACT                                646         38.990712           1541   
ACV                                540         36.881481           1267   
ACY                               1599         37.495310           3530   
ADK                                 31         18.129032             89   
ADQ                                130         16.984615            437   
AEX                               1196         30.657191           3055   
AGS                                881         30.452894           2347   
AKN                                 26         15.346154             63   
ALB                               2944         31.756114           7341   
ALO                                211         41.616114            585   
AMA                               1718         27.261350           4071   
ANC                               6247         24.352009          15819   
APN                                129         49.062016            549   
ASE                               1775         36.967324           3260   
ATL                             105603         35.611308         343076   
ATW                               1025         28.183415           2763   
AUS                              16732         33.368396          41445   
AVL                                987         34.448835           2691   
AVP                                603         32.922056           1325   
AZO                                514         36.130350           1743   
BDL                               7211         33.819720          18404   
BET                                319         21.871473            879   
BFL                               1049         30.252622           2587   
BGM                                 89         30.887640            258   
...                                ...               ...            ...   
SRQ                               1101         31.126249           3336   
STC                                 48         41.208333             77   
STL                              16807         30.976795          46273   
STT                               1623         36.191004           4306   
STX                                392         27.696429            933   
SUN                                300         27.906667            854   
SUX                                193         42.626943            591   
SWF                                272         33.525735            680   
SYR                               2105         35.327791           5475   
TLH                               1114         33.933573           3149   
TOL                                243         37.329218            901   
TPA                              23083         34.051077          63157   
TRI                                592         27.684122           1912   
TTN                               1339         47.804332           2761   
TUL                               5600         31.026429          13748   
TUS                               6103         30.320826          14956   
TVC                                850         34.183529           2666   
TWF                                337         16.670623            806   
TXK                                352         25.536932            915   
TYR                                880         31.768182           2199   
TYS                               3046        

In [103]:
#Destination Airport vs Departure Delay
DEST_AIRPORT_DEPT_DELAY_METRIC = data.loc[data['DEPARTURE_DELAY']>0,['DESTINATION_AIRPORT','DEPARTURE_DELAY']].groupby(['DESTINATION_AIRPORT']).agg(['count','mean'])
DEST_AIRPORT_DEPT_DELAY_METRIC.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


PERCENTAGE_FLIGHTS_DELAYED = data.loc[:,['DESTINATION_AIRPORT','DEPARTURE_DELAY']].groupby(['DESTINATION_AIRPORT']).count().rename(columns={"DEPARTURE_DELAY":"TOTAL_FLIGHTS"})



DEST_AIRPORT_DEPT_DELAY_METRIC = pd.merge(DEST_AIRPORT_DEPT_DELAY_METRIC,PERCENTAGE_FLIGHTS_DELAYED, how = 'inner', left_index = True, right_index = True )
DEST_AIRPORT_DEPT_DELAY_METRIC['PERCENTAGE_DELAYED'] = (DEST_AIRPORT_DEPT_DELAY_METRIC.TOTAL_DELAY_COUNT/DEST_AIRPORT_DEPT_DELAY_METRIC.TOTAL_FLIGHTS)*100
DEST_AIRPORT_DEPT_DELAY_METRIC
#AIRPORT_ARR_DELAY_METRIC.to_csv(path_or_buf='C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis',mode = 'w',sep=',')

TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  \
DESTINATION_AIRPORT                                                       
ABE                                686         37.513120           2229   
ABI                                742         31.962264           2236   
ABQ                               8417         27.856837          18996   
ABR                                134         32.417910            665   
ABY                                284         38.876761            869   
ACK                                136         43.205882            490   
ACT                                563         43.772647           1543   
ACV                                429         45.407925           1282   
ACY                               1462         40.639535           3535   
ADK                                 21         23.523810             89   
ADQ                                 82         20.609756            448   
AEX                               1020         33.952941           3068   
AGS                                763         33.913499           2349   
AKN                                 16         19.312500             63   
ALB                               3258         30.846225           7351   
ALO                                206         41.378641            588   
AMA                               1554         28.595238           4094   
ANC                               6115         24.371545          15853   
APN                                101         62.118812            556   
ASE                               1687         38.201541           3444   
ATL                             110754         34.136076         344280   
ATW                                742         35.572776           2770   
AUS                              17446         32.780924          41578   
AVL                                874         38.517162           2706   
AVP                                452         38.451327           1333   
AZO                                554         33.855596           1755   
BDL                               8039         32.968902          18450   
BET                                212         28.500000            887   
BFL                                786         39.162850           2601   
BGM                                 66         41.712121            260   
...                                ...               ...            ...   
SRQ                               1194         29.384422           3340   
STC                                 41         42.097561             77   
STL                              18497         29.881873          46384   
STT                               1689         35.483718           4327   
STX                                368         28.866848            936   
SUN                                263         31.745247            953   
SUX                                215         37.102326            593   
SWF                                282         35.336879            682   
SYR                               1988         37.165996           5484   
TLH                               1055         36.300474           3157   
TOL                                275         31.400000            903   
TPA                              25990         31.224740          63346   
TRI                                544         30.628676           1912   
TTN                               1148         52.695122           2772   
TUL                               5575         31.358027          13779   
TUS                               5892         31.841310          14991   
TVC                                839         33.069130           2675   
TWF                                101         31.673267            811   
TXK                                310         26.474194            927   
TYR                                696         36.352011           2209   
TYS                               2518        

In [89]:
#Destination Airport vs Arrival Delay
DEST_AIRPORT_ARR_DELAY_METRIC = data.loc[data['ARRIVAL_DELAY']>0,['DESTINATION_AIRPORT','ARRIVAL_DELAY']].groupby(['DESTINATION_AIRPORT']).agg(['count','mean'])
DEST_AIRPORT_ARR_DELAY_METRIC.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


PERCENTAGE_FLIGHTS_DELAYED = data.loc[:,['DESTINATION_AIRPORT','ARRIVAL_DELAY']].groupby(['DESTINATION_AIRPORT']).count().rename(columns={"ARRIVAL_DELAY":"TOTAL_FLIGHTS"})



DEST_AIRPORT_ARR_DELAY_METRIC = pd.merge(DEST_AIRPORT_ARR_DELAY_METRIC,PERCENTAGE_FLIGHTS_DELAYED, how = 'inner', left_index = True, right_index = True )
DEST_AIRPORT_ARR_DELAY_METRIC['PERCENTAGE_DELAYED'] = (DEST_AIRPORT_ARR_DELAY_METRIC.TOTAL_DELAY_COUNT/DEST_AIRPORT_ARR_DELAY_METRIC.TOTAL_FLIGHTS)*100
DEST_AIRPORT_ARR_DELAY_METRIC
#AIRPORT_ARR_DELAY_METRIC.to_csv(path_or_buf='C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis',mode = 'w',sep=',')

27413

In [104]:
#Day Of The Week vs Arrival Delay
DAY_OF_WEEK_ARR_DELAY_METRIC = data.loc[data['ARRIVAL_DELAY']>0,['DAY_OF_WEEK','ARRIVAL_DELAY']].groupby(['DAY_OF_WEEK']).agg(['count','mean'])
DAY_OF_WEEK_ARR_DELAY_METRIC.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


PERCENTAGE_FLIGHTS_DELAYED = data.loc[:,['DAY_OF_WEEK','ARRIVAL_DELAY']].groupby(['DAY_OF_WEEK']).count().rename(columns={"ARRIVAL_DELAY":"TOTAL_FLIGHTS"})



DAY_OF_WEEK_ARR_DELAY_METRIC = pd.merge(DAY_OF_WEEK_ARR_DELAY_METRIC,PERCENTAGE_FLIGHTS_DELAYED, how = 'inner', left_index = True, right_index = True )
DAY_OF_WEEK_ARR_DELAY_METRIC['PERCENTAGE_DELAYED'] = (DAY_OF_WEEK_ARR_DELAY_METRIC.TOTAL_DELAY_COUNT/DAY_OF_WEEK_ARR_DELAY_METRIC.TOTAL_FLIGHTS)*100
DAY_OF_WEEK_ARR_DELAY_METRIC
#AIRPORT_ARR_DELAY_METRIC.to_csv(path_or_buf='C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis',mode = 'w',sep=',')

TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  \
DAY_OF_WEEK                                                       
1                       297746         36.757303         775714   
2                       280846         34.471689         763813   
3                       287420         32.302832         778658   
4                       309277         33.351119         775433   
5                       296082         31.237350         770324   
6                       207521         32.363587         627502   
7                       266754         34.134690         739686   

             PERCENTAGE_DELAYED  
DAY_OF_WEEK                      
1                     38.383476  
2                     36.768947  
3                     36.912226  
4                     39.884426  
5                     38.436035  
6                     33.070970  
7                     36.063140

In [105]:
#Day Of The Week vs Departure Delay
DAY_OF_WEEK_DEPT_DELAY_METRIC = data.loc[data['DEPARTURE_DELAY']>0,['DAY_OF_WEEK','DEPARTURE_DELAY']].groupby(['DAY_OF_WEEK']).agg(['count','mean'])
DAY_OF_WEEK_DEPT_DELAY_METRIC.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


PERCENTAGE_FLIGHTS_DELAYED = data.loc[:,['DAY_OF_WEEK','DEPARTURE_DELAY']].groupby(['DAY_OF_WEEK']).count().rename(columns={"DEPARTURE_DELAY":"TOTAL_FLIGHTS"})



DAY_OF_WEEK_DEPT_DELAY_METRIC = pd.merge(DAY_OF_WEEK_DEPT_DELAY_METRIC,PERCENTAGE_FLIGHTS_DELAYED, how = 'inner', left_index = True, right_index = True )
DAY_OF_WEEK_DEPT_DELAY_METRIC['PERCENTAGE_DELAYED'] = (DAY_OF_WEEK_DEPT_DELAY_METRIC.TOTAL_DELAY_COUNT/DAY_OF_WEEK_DEPT_DELAY_METRIC.TOTAL_FLIGHTS)*100
DAY_OF_WEEK_DEPT_DELAY_METRIC
#AIRPORT_ARR_DELAY_METRIC.to_csv(path_or_buf='C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis',mode = 'w',sep=',')

TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  \
DAY_OF_WEEK                                                       
1                       305229         35.946725         779131   
2                       281252         34.420786         766349   
3                       285819         32.469643         780921   
4                       307950         32.990989         778126   
5                       299475         31.064727         772459   
6                       219884         31.518110         629889   
7                       280907         32.917051         742225   

             PERCENTAGE_DELAYED  
DAY_OF_WEEK                      
1                     39.175569  
2                     36.700250  
3                     36.600245  
4                     39.575853  
5                     38.769048  
6                     34.908373  
7                     37.846610

In [136]:
# Month Vs Arrival Delay

#data.index = data['FLIGHT_DATE']
MONTHWISE_ARR_DELAY_METRICS = data.loc[data['ARRIVAL_DELAY']>0,['FLIGHT_DATE','ARRIVAL_DELAY']].groupby(pd.TimeGrouper(freq='M')).agg(['count','mean'])
MONTHWISE_ARR_DELAY_METRICS.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


MONTHLY_TOTAL_FLIGHTS = data.loc[:,['ARRIVAL_DELAY']].groupby(pd.TimeGrouper(freq='M')).agg(['count'])
MONTHLY_TOTAL_FLIGHTS.columns = ['TOTAL_FLIGHTS']


MONTHWISE_ARR_DELAY_METRICS = pd.merge(MONTHWISE_ARR_DELAY_METRICS,MONTHLY_TOTAL_FLIGHTS,how = 'inner',left_index = True, right_index=True)
MONTHWISE_ARR_DELAY_METRICS['PERCENTAGE_DELAYS'] = (MONTHWISE_ARR_DELAY_METRICS.TOTAL_DELAY_COUNT/MONTHWISE_ARR_DELAY_METRICS.TOTAL_FLIGHTS)*100
MONTHWISE_ARR_DELAY_METRICS


TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  \
FLIGHT_DATE                                                       
2015-01-31              183110         32.755759         457013   
2015-02-28              175443         35.084894         407663   
2015-03-31              190133         31.764549         492138   
2015-04-30              171820         30.157083         479251   
2015-05-31              175178         34.111806         489641   
2015-06-30              206989         38.423327         492847   
2015-07-31              199717         34.889674         514384   
2015-08-31              180891         34.050207         503956   
2015-09-30              133432         28.015738         462153   
2015-10-31                   0               NaN              0   
2015-11-30              149439         30.351220         462367   
2015-12-31              179494         36.718654         469717   

             PERCENTAGE_DELAYS  
FLIGHT_DATE                     
2015-01-31           40.066694  
2015-02-28           43.036282  
2015-03-31           38.634082  
2015-04-30           35.851777  
2015-05-31           35.776824  
2015-06-30           41.998632  
2015-07-31           38.826441  
2015-08-31           35.894205  
2015-09-30           28.871824  
2015-10-31                 NaN  
2015-11-30           32.320429  
2015-12-31           38.213222

In [141]:
# Month Vs Departure Delay

#data.index = data['FLIGHT_DATE']
MONTHWISE_DEPT_DELAY_METRICS = data.loc[data['DEPARTURE_DELAY']>0,['FLIGHT_DATE','DEPARTURE_DELAY']].groupby(pd.TimeGrouper(freq='M')).agg(['count','mean'])
MONTHWISE_DEPT_DELAY_METRICS.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']


MONTHLY_TOTAL_FLIGHTS = data.loc[:,['DEPARTURE_DELAY']].groupby(pd.TimeGrouper(freq='M')).agg(['count'])
MONTHLY_TOTAL_FLIGHTS.columns = ['TOTAL_FLIGHTS']


MONTHWISE_DEPT_DELAY_METRICS = pd.merge(MONTHWISE_DEPT_DELAY_METRICS,MONTHLY_TOTAL_FLIGHTS,how = 'inner',left_index = True, right_index=True)
MONTHWISE_DEPT_DELAY_METRICS['PERCENTAGE_DELAYS'] = (MONTHWISE_DEPT_DELAY_METRICS.TOTAL_DELAY_COUNT/MONTHWISE_DEPT_DELAY_METRICS.TOTAL_FLIGHTS)*100
MONTHWISE_DEPT_DELAY_METRICS


TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  \
FLIGHT_DATE                                                       
2015-01-31              176627         32.623200         458311   
2015-02-28              173442         33.960667         409132   
2015-03-31              193817         31.354685         493673   
2015-04-30              167314         30.690080         480898   
2015-05-31              178856         33.704052         491657   
2015-06-30              215381         37.546367         495199   
2015-07-31              209619         34.204261         516211   
2015-08-31              190840         33.326425         505817   
2015-09-30              132591         28.099939         463018   
2015-10-31                   0               NaN              0   
2015-11-30              152690         29.983660         463633   
2015-12-31              189339         35.702407         471551   

             PERCENTAGE_DELAYS  
FLIGHT_DATE                     
2015-01-31           38.538678  
2015-02-28           42.392675  
2015-03-31           39.260199  
2015-04-30           34.791993  
2015-05-31           36.378207  
2015-06-30           43.493828  
2015-07-31           40.607232  
2015-08-31           37.729060  
2015-09-30           28.636252  
2015-10-31                 NaN  
2015-11-30           32.933376  
2015-12-31           40.152391

In [158]:
# State Vs Arrival Delay(Origin Airport)
STATE_ARR_DELAY_METRICS = pd.merge(data.loc[data['ARRIVAL_DELAY']>0,['ORIGIN_AIRPORT','ARRIVAL_DELAY']],airport_data.loc[:,['IATA_CODE','STATE']],how='outer',left_on =['ORIGIN_AIRPORT'],right_on=['IATA_CODE'])
STATE_ARR_DELAY_METRICS = STATE_ARR_DELAY_METRICS.loc[:,['STATE','ARRIVAL_DELAY']].groupby(['STATE']).agg(['count','mean'])
STATE_ARR_DELAY_METRICS.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']

STATE_TOTAL_FLIGHTS = pd.merge(data.loc[:,['ORIGIN_AIRPORT','ARRIVAL_DELAY']],airport_data.loc[:,['IATA_CODE','STATE']], how='outer',left_on =['ORIGIN_AIRPORT'],right_on=['IATA_CODE'])
STATE_TOTAL_FLIGHTS = STATE_TOTAL_FLIGHTS.loc[:,['STATE','ARRIVAL_DELAY']].groupby(['STATE']).agg(['count'])
STATE_TOTAL_FLIGHTS.columns = ['TOTAL_FLIGHTS']
STATE_TOTAL_FLIGHTS

STATE_ARR_DELAY_METRICS = pd.merge(STATE_ARR_DELAY_METRICS,STATE_TOTAL_FLIGHTS, how = 'inner', left_index=True, right_index=True)
STATE_ARR_DELAY_METRICS['PERCENTAGE_DELAYS'] = (STATE_ARR_DELAY_METRICS.TOTAL_DELAY_COUNT/STATE_ARR_DELAY_METRICS.TOTAL_FLIGHTS)*100
STATE_ARR_DELAY_METRICS


TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  PERCENTAGE_DELAYS
STATE                                                                       
AK                 11533         25.894390          32754          35.210967
AL                  9455         34.768800          25960          36.421418
AR                  8366         40.304208          21923          38.160836
AS                    67         38.358209            106          63.207547
AZ                 64448         27.121990         163938          39.312423
CA                248922         30.373153         638357          38.994168
CO                 87695         33.588620         212126          41.340995
CT                  5691         34.367071          18375          30.971429
DE                    53         57.207547             95          55.789474
FL                148633         36.680226         409611          36.286379
GA                119817         31.254630         356943          33.567544
GU                   136         31.867647            331          41.087613
HI                 32306         19.295827          90670          35.630308
IA                  6044         42.671410          16728          36.131038
ID                  5465         32.990485          16661          32.801152
IL                150245         38.031682         369479          40.664016
IN                 12881         38.252387          38177          33.740210
KS                  3766         39.406798          10633          35.418038
KY                 12250         38.414612          35829          34.190181
LA                 22957         35.926820          63944          35.901727
MA                 39180         35.642956         106208          36.889876
MD                 34979         34.148375          84329          41.479206
ME                  1584         41.312500           4892          32.379395
MI                 44669         35.927444         134584          33.190424
MN                 39831         30.407773         117449          33.913443
MO                 33440         33.166866          92192          36.272128
MS                  4261         37.327388          12782          33.335941
MT                  3853         30.828186          14675          26.255537
NC                 57276         31.255238         144394          39.666468
ND                  5406         37.096744          16088          33.602685
NE                  6636         35.085142          19613          33.834701
NH                  1771         35.457369           6115          28.961570
NJ                 38566         39.938677         104639          36.856239
NM                  8009         33.068298          21815          36.713271
NV                 57789         31.624617         145305          39.770827
NY                 85965         41.491130         237916          36.132501
OH                 23478         38.014482          72555          32.358900
OK                 10531         36.305859          30734          34.264983
OR                 17659         29.248825          55148          32.021107
PA                 34263         37.229285          98018          34.955824
PR                  8642         36.879773          26468          32.650748
RI                  3297         33.889900          10757          30.649809
SC                  9454         37.206685          27679          34.155858
SD                  3472         37.116359           9017          38.505046
TN                 26390         34.892118          73537          35.886696
TX                245912         32.822266         616518          39.887238
UT                 31843         25.892410          99252          32.082981
VA                 46518         38.799561         134752          34.521194
VI                  1719         43.152414           5103          33.686067
VT                  1044         43.052682           2884          36.199723
WA 

In [161]:
# State Vs Departure Delay(Origin Airport)
STATE_DEPT_DELAY_METRICS = pd.merge(data.loc[data['DEPARTURE_DELAY']>0,['ORIGIN_AIRPORT','DEPARTURE_DELAY']],airport_data.loc[:,['IATA_CODE','STATE']],how='outer',left_on =['ORIGIN_AIRPORT'],right_on=['IATA_CODE'])
STATE_DEPT_DELAY_METRICS = STATE_DEPT_DELAY_METRICS.loc[:,['STATE','DEPARTURE_DELAY']].groupby(['STATE']).agg(['count','mean'])
STATE_DEPT_DELAY_METRICS.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']

STATE_TOTAL_FLIGHTS = pd.merge(data.loc[:,['ORIGIN_AIRPORT','DEPARTURE_DELAY']],airport_data.loc[:,['IATA_CODE','STATE']], how='outer',left_on =['ORIGIN_AIRPORT'],right_on=['IATA_CODE'])
STATE_TOTAL_FLIGHTS = STATE_TOTAL_FLIGHTS.loc[:,['STATE','DEPARTURE_DELAY']].groupby(['STATE']).agg(['count'])
STATE_TOTAL_FLIGHTS.columns = ['TOTAL_FLIGHTS']
STATE_TOTAL_FLIGHTS

STATE_DEPT_DELAY_METRICS = pd.merge(STATE_DEPT_DELAY_METRICS,STATE_TOTAL_FLIGHTS, how = 'inner', left_index=True, right_index=True)
STATE_DEPT_DELAY_METRICS['PERCENTAGE_DELAYS'] = (STATE_DEPT_DELAY_METRICS.TOTAL_DELAY_COUNT/STATE_DEPT_DELAY_METRICS.TOTAL_FLIGHTS)*100
STATE_DEPT_DELAY_METRICS


TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  PERCENTAGE_DELAYS
STATE                                                                       
AK                 11142         26.160115          32987          33.776942
AL                  9373         31.830791          26106          35.903624
AR                  8616         33.780525          22077          39.027042
AS                    45         28.244444            107          42.056075
AZ                 60968         28.084553         164135          37.145033
CA                270150         30.755699         640267          42.193335
CO                 85350         33.965366         212823          40.103748
CT                  8039         32.968902          18450          43.571816
DE                    40         59.625000             95          42.105263
FL                163647         32.453195         411236          39.793938
GA                115155         34.147202         357767          32.187150
GU                   241         32.535270            332          72.590361
HI                 27647         23.592108          90937          30.402366
IA                  6334         37.027787          16879          37.525920
ID                  5706         31.234665          16812          33.940043
IL                136729         38.181483         370419          36.911983
IN                 13893         34.789174          38376          36.202314
KS                  4015         32.360149          10722          37.446372
KY                 12848         33.385118          36070          35.619629
LA                 24334         31.978549          64248          37.875109
MA                 42371         35.100564         106679          39.718220
MD                 34870         29.552767          84637          41.199475
ME                  1883         36.291556           4934          38.163762
MI                 43645         36.406095         134872          32.360312
MN                 37704         34.021854         117744          32.022014
MO                 37670         30.429121          92595          40.682542
MS                  4126         34.410810          12859          32.086476
MT                  4160         30.387740          14765          28.174738
NC                 46968         29.943195         144663          32.467182
ND                  4910         34.531568          16262          30.193088
NE                  7586         31.807145          19724          38.460758
NH                  2695         32.376252           6143          43.871073
NJ                 46704         38.284494         104876          44.532591
NM                  9383         28.134925          21917          42.811516
NV                 60182         28.743894         145799          41.277375
NY                 97375         40.077330         238971          40.747622
OH                 28031         34.607684          72936          38.432324
OK                 12746         32.034050          30916          41.227843
OR                 20305         30.098498          55302          36.716574
PA                 37691         33.645645          98367          38.316712
PR                 12143         30.591946          26572          45.698480
RI                  4450         31.841798          10825          41.108545
SC                  9946         33.075809          27834          35.733276
SD                  3077         34.821254           9099          33.816903
TN                 28861         30.762586          73952          39.026666
TX                229420         34.875072         618279          37.106226
UT                 28835         30.116802          99475          28.987183
VA                 51976         32.660574         135406          38.385301
VI                  2057         34.299951           5263          39.084173
VT                  1085         42.209217           2912          37.259615
WA 

In [162]:
# State Vs Departure Delay(Destination Airport)

STATE_DEPT_DELAY_METRICS = pd.merge(data.loc[data['DEPARTURE_DELAY']>0,['DESTINATION_AIRPORT','DEPARTURE_DELAY']],airport_data.loc[:,['IATA_CODE','STATE']],how='outer',left_on =['DESTINATION_AIRPORT'],right_on=['IATA_CODE'])
STATE_DEPT_DELAY_METRICS = STATE_DEPT_DELAY_METRICS.loc[:,['STATE','DEPARTURE_DELAY']].groupby(['STATE']).agg(['count','mean'])
STATE_DEPT_DELAY_METRICS.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']

STATE_TOTAL_FLIGHTS = pd.merge(data.loc[:,['DESTINATION_AIRPORT','DEPARTURE_DELAY']],airport_data.loc[:,['IATA_CODE','STATE']], how='outer',left_on =['DESTINATION_AIRPORT'],right_on=['IATA_CODE'])
STATE_TOTAL_FLIGHTS = STATE_TOTAL_FLIGHTS.loc[:,['STATE','DEPARTURE_DELAY']].groupby(['STATE']).agg(['count'])
STATE_TOTAL_FLIGHTS.columns = ['TOTAL_FLIGHTS']
STATE_TOTAL_FLIGHTS

STATE_DEPT_DELAY_METRICS = pd.merge(STATE_DEPT_DELAY_METRICS,STATE_TOTAL_FLIGHTS, how = 'inner', left_index=True, right_index=True)
STATE_DEPT_DELAY_METRICS['PERCENTAGE_DELAYS'] = (STATE_DEPT_DELAY_METRICS.TOTAL_DELAY_COUNT/STATE_DEPT_DELAY_METRICS.TOTAL_FLIGHTS)*100
STATE_DEPT_DELAY_METRICS


TOTAL_DELAY_COUNT  MEAN_DELAY_COUNT  TOTAL_FLIGHTS  PERCENTAGE_DELAYS
STATE                                                                       
AK                 11142         26.160115          32987          33.776942
AL                  9373         31.830791          26106          35.903624
AR                  8616         33.780525          22077          39.027042
AS                    45         28.244444            107          42.056075
AZ                 60968         28.084553         164135          37.145033
CA                270150         30.755699         640267          42.193335
CO                 85350         33.965366         212823          40.103748
CT                  8039         32.968902          18450          43.571816
DE                    40         59.625000             95          42.105263
FL                163647         32.453195         411236          39.793938
GA                115155         34.147202         357767          32.187150
GU                   241         32.535270            332          72.590361
HI                 27647         23.592108          90937          30.402366
IA                  6334         37.027787          16879          37.525920
ID                  5706         31.234665          16812          33.940043
IL                136729         38.181483         370419          36.911983
IN                 13893         34.789174          38376          36.202314
KS                  4015         32.360149          10722          37.446372
KY                 12848         33.385118          36070          35.619629
LA                 24334         31.978549          64248          37.875109
MA                 42371         35.100564         106679          39.718220
MD                 34870         29.552767          84637          41.199475
ME                  1883         36.291556           4934          38.163762
MI                 43645         36.406095         134872          32.360312
MN                 37704         34.021854         117744          32.022014
MO                 37670         30.429121          92595          40.682542
MS                  4126         34.410810          12859          32.086476
MT                  4160         30.387740          14765          28.174738
NC                 46968         29.943195         144663          32.467182
ND                  4910         34.531568          16262          30.193088
NE                  7586         31.807145          19724          38.460758
NH                  2695         32.376252           6143          43.871073
NJ                 46704         38.284494         104876          44.532591
NM                  9383         28.134925          21917          42.811516
NV                 60182         28.743894         145799          41.277375
NY                 97375         40.077330         238971          40.747622
OH                 28031         34.607684          72936          38.432324
OK                 12746         32.034050          30916          41.227843
OR                 20305         30.098498          55302          36.716574
PA                 37691         33.645645          98367          38.316712
PR                 12143         30.591946          26572          45.698480
RI                  4450         31.841798          10825          41.108545
SC                  9946         33.075809          27834          35.733276
SD                  3077         34.821254           9099          33.816903
TN                 28861         30.762586          73952          39.026666
TX                229420         34.875072         618279          37.106226
UT                 28835         30.116802          99475          28.987183
VA                 51976         32.660574         135406          38.385301
VI                  2057         34.299951           5263          39.084173
VT                  1085         42.209217           2912          37.259615
WA 

In [200]:
# State Vs Arrival Delay(Destination Airport)

STATE_ARR_DELAY_METRICS = pd.merge(data.loc[data['ARRIVAL_DELAY']>0,['DESTINATION_AIRPORT','ARRIVAL_DELAY']],airport_data.loc[:,['IATA_CODE','STATE']],how='outer',left_on =['DESTINATION_AIRPORT'],right_on=['IATA_CODE'])
STATE_ARR_DELAY_METRICS = STATE_ARR_DELAY_METRICS.loc[:,['STATE','ARRIVAL_DELAY']].groupby(['STATE']).agg(['count','mean'])
STATE_ARR_DELAY_METRICS.columns = ['TOTAL_DELAY_COUNT','MEAN_DELAY_COUNT']

STATE_TOTAL_FLIGHTS = pd.merge(data.loc[:,['DESTINATION_AIRPORT','ARRIVAL_DELAY']],airport_data.loc[:,['IATA_CODE','STATE']], how='outer',left_on =['DESTINATION_AIRPORT'],right_on=['IATA_CODE'])
STATE_TOTAL_FLIGHTS = STATE_TOTAL_FLIGHTS.loc[:,['STATE','ARRIVAL_DELAY']].groupby(['STATE']).agg(['count'])
STATE_TOTAL_FLIGHTS.columns = ['TOTAL_FLIGHTS']
STATE_TOTAL_FLIGHTS

STATE_ARR_DELAY_METRICS = pd.merge(STATE_ARR_DELAY_METRICS,STATE_TOTAL_FLIGHTS, how = 'inner', left_index=True, right_index=True)
STATE_ARR_DELAY_METRICS['PERCENTAGE_DELAYS'] = (STATE_ARR_DELAY_METRICS.TOTAL_DELAY_COUNT/STATE_ARR_DELAY_METRICS.TOTAL_FLIGHTS)*100
STATE_ARR_DELAY_METRICS.to_csv ('C:\\Users\\rashm\\PycharmProjects\\DataProgrammingProject\\ExploratoryAnalysis\\STATE_ARR_DELAY_METRICS_BY_DESTINATION.csv')


In [28]:
holiday_data = pd.read_csv('Holiday_List.csv',delimiter=',')
holiday_data.head()

DAY_TYPE  MONTH  DAY  YEAR
0        P      1    1  2015
1        P      1    2  2015
2        P      1    3  2015
3        P      1   12  2015
4        P      2   12  2015

In [29]:
holiday_data['FLIGHT_DATE'] = pd.to_datetime(holiday_data[['YEAR','MONTH','DAY']])

holiday_data = holiday_data.drop(['YEAR','MONTH','DAY'],axis=1)
holiday_data.head()

DAY_TYPE FLIGHT_DATE
0        P  2015-01-01
1        P  2015-01-02
2        P  2015-01-03
3        P  2015-01-12
4        P  2015-02-12

In [33]:
data = pd.merge(data,holiday_data, how = 'left', on='FLIGHT_DATE')
data.head()
data.DAY_TYPE = data.DAY_TYPE.fillna('NP')
data.loc[data.FLIGHT_DATE == '2015-01-10',['DAY_TYPE']]


DAY_TYPE
137658       NP
137659       NP
137660       NP
137661       NP
137662       NP
137663       NP
137664       NP
137665       NP
137666       NP
137667       NP
137668       NP
137669       NP
137670       NP
137671       NP
137672       NP
137673       NP
137674       NP
137675       NP
137676       NP
137677       NP
137678       NP
137679       NP
137680       NP
137681       NP
137682       NP
137683       NP
137684       NP
137685       NP
137686       NP
137687       NP
...         ...
149766       NP
149767       NP
149768       NP
149769       NP
149770       NP
149771       NP
149772       NP
149773       NP
149774       NP
149775       NP
149776       NP
149777       NP
149778       NP
149779       NP
149780       NP
149781       NP
149782       NP
149783       NP
149784       NP
149785       NP
149786       NP
149787       NP
149788       NP
149789       NP
149790       NP
149791       NP
149792       NP
149793       NP
149794       NP
149795       NP

[12138 rows x 1 columns]

In [44]:
 data.to_csv('data_withpeakdetails.csv')

In [39]:
from __future__ import division

In [40]:
#Departure Delay Vs Day-Type(pean and non-peak days)

PEAK_DAY_DELAY_METRICS = data.loc[data.DEPARTURE_DELAY > 15,['DEPARTURE_DELAY','DAY_TYPE']].groupby(['DAY_TYPE']).agg(['mean','median','count'])
PEAK_DAY_DELAY_METRICS.columns = ['MEAN_DELAY','MEDIAN_DELAY','TOTAL_FLIGHT_DELAYED']

TOTAL_FLIGHTS = data.loc[:,['DEPARTURE_DELAY','DAY_TYPE']].groupby(['DAY_TYPE']).count()
TOTAL_FLIGHTS.columns = ['TOTAL_FLIGHTS']

PEAK_DAY_DELAY_METRICS = pd.merge(PEAK_DAY_DELAY_METRICS,TOTAL_FLIGHTS, how = 'left', left_index = True, right_index = True)
PEAK_DAY_DELAY_METRICS['PERCENTAGE_DELAY'] = ((PEAK_DAY_DELAY_METRICS.TOTAL_FLIGHT_DELAYED)/(PEAK_DAY_DELAY_METRICS.TOTAL_FLIGHTS))*100
PEAK_DAY_DELAY_METRICS


MEAN_DELAY  MEDIAN_DELAY  TOTAL_FLIGHT_DELAYED  TOTAL_FLIGHTS  \
DAY_TYPE                                                                  
NP         61.188584          40.0                705422        3984607   
P          62.108445          40.0                248707        1246523   

          PERCENTAGE_DELAY  
DAY_TYPE                    
NP               17.703678  
P                19.952059

In [56]:
data.head()

DAY_OF_WEEK AIRLINE ORIGIN_AIRPORT DESTINATION_AIRPORT SCHEDULED_DEPARTURE  \
0            4      AS            ANC                 SEA 2015-01-01 00:05:00   
1            4      AA            LAX                 PBI 2015-01-01 00:10:00   
2            4      US            SFO                 CLT 2015-01-01 00:20:00   
3            4      AA            LAX                 MIA 2015-01-01 00:20:00   
4            4      AS            SEA                 ANC 2015-01-01 00:25:00   

  DEPARTURE_TIME  DEPARTURE_DELAY  SCHEDULED_TIME  ELAPSED_TIME  AIR_TIME  \
0       23:54:00            -11.0           205.0         194.0     169.0   
1       00:02:00             -8.0           280.0         279.0     263.0   
2       00:18:00             -2.0           286.0         293.0     266.0   
3       00:15:00             -5.0           285.0         281.0     258.0   
4       00:24:00             -1.0           235.0         215.0     199.0   

   DISTANCE SCHEDULED_ARRIVAL ARRIVAL_TIME  ARRIVAL_DELAY FLIGHT_DATE  \
0      1448          04:30:00     04:08:00          -22.0  2015-01-01   
1      2330          07:50:00     07:41:00           -9.0  2015-01-01   
2      2296          08:06:00     08:11:00            5.0  2015-01-01   
3      2342          08:05:00     07:56:00           -9.0  2015-01-01   
4      1448          03:20:00     02:59:00          -21.0  2015-01-01   

   DAY_TYPE  AIRLINE_ENCODE  ORIGIN_AIRPORT_ENCODE  DESTINATION_AIRPORT_ENCODE  
0         1               1                     17                         277  
1         1               0                    176                         235  
2         1              11                    278                          66  
3         1               0                    176                         203  
4         1               1                    277                          17

In [10]:
#data = pd.read_csv('data_withpeakdetails.csv', delimiter = ',')


Unnamed: 0  DAY_OF_WEEK AIRLINE ORIGIN_AIRPORT DESTINATION_AIRPORT  \
0           0            4      AS            ANC                 SEA   
1           1            4      AA            LAX                 PBI   
2           2            4      US            SFO                 CLT   
3           3            4      AA            LAX                 MIA   
4           4            4      AS            SEA                 ANC   

   SCHEDULED_DEPARTURE DEPARTURE_TIME  DEPARTURE_DELAY  SCHEDULED_TIME  \
0  2015-01-01 00:05:00       23:54:00            -11.0           205.0   
1  2015-01-01 00:10:00       00:02:00             -8.0           280.0   
2  2015-01-01 00:20:00       00:18:00             -2.0           286.0   
3  2015-01-01 00:20:00       00:15:00             -5.0           285.0   
4  2015-01-01 00:25:00       00:24:00             -1.0           235.0   

   ELAPSED_TIME  AIR_TIME  DISTANCE SCHEDULED_ARRIVAL ARRIVAL_TIME  \
0         194.0     169.0      1448          04:30:00     04:08:00   
1         279.0     263.0      2330          07:50:00     07:41:00   
2         293.0     266.0      2296          08:06:00     08:11:00   
3         281.0     258.0      2342          08:05:00     07:56:00   
4         215.0     199.0      1448          03:20:00     02:59:00   

   ARRIVAL_DELAY          FLIGHT_DATE  DAY_TYPE  AIRLINE_ENCODE  \
0          -22.0  2015-01-01 00:00:00         1               1   
1           -9.0  2015-01-01 00:00:00         1               0   
2            5.0  2015-01-01 00:00:00         1              11   
3           -9.0  2015-01-01 00:00:00         1               0   
4          -21.0  2015-01-01 00:00:00         1               1   

   ORIGIN_AIRPORT_ENCODE  DESTINATION_AIRPORT_ENCODE  
0                     17                         277  
1                    176                         235  
2                    278                          66  
3                    176                         203  
4                    277                          17

In [8]:
le = preprocessing.LabelEncoder()
data['AIRLINE_ENCODE'] = le.fit_transform(data['AIRLINE'])
data['ORIGIN_AIRPORT_ENCODE'] = le.fit_transform(data['ORIGIN_AIRPORT'])
data['DESTINATION_AIRPORT_ENCODE'] = le.fit_transform(data['DESTINATION_AIRPORT'])
data['DAY_TYPE'] = le.fit_transform(data['DAY_TYPE'])

In [11]:
train, test = train_test_split(data, test_size = 0.25)
model = LinearSVC(dual = False,random_state=0) 
X_train = data[['DAY_OF_WEEK','AIRLINE_ENCODE','ORIGIN_AIRPORT_ENCODE','DESTINATION_AIRPORT_ENCODE','AIR_TIME','DISTANCE','DAY_TYPE']]
y_train = data['DEPARTURE_DELAY']
X_test = data[['DAY_OF_WEEK','AIRLINE_ENCODE','ORIGIN_AIRPORT_ENCODE','DESTINATION_AIRPORT_ENCODE','AIR_TIME','DISTANCE','DAY_TYPE']]
y_test = data['DEPARTURE_DELAY']




In [ ]:
model.fit(X_train, y_train)


In [ ]:
#Predict Output
predicted = model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, predicted)
print cm
print accuracy_score(y_test, y_pred)